In [1]:
from langchain_community.document_loaders import PDFMinerLoader
import pandas as pd
import os
import re
import sys
#sys.path.append('d:\\Local-RAG-Assistant-Chatbot\\')
from datasets import load_dataset

d:\machineLearning\miniconda\envs\env_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset('squad', split='validation')
data_dict = {
    'id': [row['id'] for row in dataset],
    'title': [row['title'] for row in dataset],
    'context': [row['context'] for row in dataset],
    'question': [row['question'] for row in dataset],
    'answers': [row['answers'] for row in dataset]
}

df = pd.DataFrame(data_dict)
df = df[df['title'] == 'Computational_complexity_theory']

In [3]:
df

,id,title,context,question,answers
1680,56e16182e3433e1400422e28,Computational_complexity_theory,Computational complexity theory is a branch of...,What branch of theoretical computer science de...,"{'text': ['Computational complexity theory', '..."
1681,56e16182e3433e1400422e29,Computational_complexity_theory,Computational complexity theory is a branch of...,By what main attribute are computational probl...,"{'text': ['inherent difficulty', 'their inhere..."
1682,56e16182e3433e1400422e2a,Computational_complexity_theory,Computational complexity theory is a branch of...,What is the term for a task that generally len...,"{'text': ['computational problems', 'A computa..."
1683,56e16839cd28a01900c67887,Computational_complexity_theory,A problem is regarded as inherently difficult ...,What measure of a computational problem broadl...,{'text': ['if its solution requires significan...
1684,56e16839cd28a01900c67888,Computational_complexity_theory,A problem is regarded as inherently difficult ...,What method is used to intuitively assess or q...,{'text': ['mathematical models of computation'...
...,...,...,...,...,...
1872,56e20a27cd28a01900c67c25,Computational_complexity_theory,Even though some proofs of complexity-theoreti...,In the effort of maintaining a level of abstra...,"{'text': ['encoding', 'encoding', 'encoding'],..."
1873,56e20a3ae3433e140042324a,Computational_complexity_theory,"In 1967, Manuel Blum developed an axiomatic co...",Who is responsible for axiomatic complexity th...,"{'text': ['Manuel Blum', 'Manuel Blum', 'Manue..."
1874,56e20a3ae3433e140042324b,Computational_complexity_theory,"In 1967, Manuel Blum developed an axiomatic co...",What theorem was implicated by Manuel Blum's a...,"{'text': ['speed-up theorem', 'speed-up theore..."
1875,56e20a3ae3433e140042324d,Computational_complexity_theory,"In 1967, Manuel Blum developed an axiomatic co...",What is the paper written by Richard Karp in 1...,"{'text': ['""Reducibility Among Combinatorial P..."


In [4]:
df['first_answer'] = df['answers'].apply(lambda x: x['text'][1])

In [5]:
df = df.drop_duplicates(subset='context', keep='first')

In [6]:
df

,id,title,context,question,answers,first_answer
1680,56e16182e3433e1400422e28,Computational_complexity_theory,Computational complexity theory is a branch of...,What branch of theoretical computer science de...,"{'text': ['Computational complexity theory', '...",Computational complexity theory
1683,56e16839cd28a01900c67887,Computational_complexity_theory,A problem is regarded as inherently difficult ...,What measure of a computational problem broadl...,{'text': ['if its solution requires significan...,its solution requires significant resources
1688,56e17644e3433e1400422f40,Computational_complexity_theory,Closely related fields in theoretical computer...,What two fields of theoretical computer scienc...,{'text': ['analysis of algorithms and computab...,analysis of algorithms and computability theory
1692,56e17a7ccd28a01900c679a1,Computational_complexity_theory,A computational problem can be viewed as an in...,What is the name given to the input string of ...,"{'text': ['problem instance', 'a problem insta...",a problem instance
1697,56e17e6ee3433e1400422f7f,Computational_complexity_theory,To further highlight the difference between a ...,By how many kilometers does the traveling sale...,"{'text': ['2000', '2000', '2000'], 'answer_sta...",2000
1700,56e181d9e3433e1400422fa0,Computational_complexity_theory,"When considering computational problems, a pro...","In a computational problem, what can be descri...","{'text': ['problem instance', 'a problem insta...",a problem instance
1705,56e190bce3433e1400422fc8,Computational_complexity_theory,Decision problems are one of the central objec...,What kind of problems are one of the main topi...,"{'text': ['Decision problems', 'Decision probl...",Decision problems
1710,56e19557e3433e1400422fee,Computational_complexity_theory,An example of a decision problem is the follow...,What kind of graph is an example of an input u...,"{'text': ['arbitrary graph', 'arbitrary', 'arb...",arbitrary
1713,56e19724cd28a01900c679f6,Computational_complexity_theory,A function problem is a computational problem ...,A function problem is an example of what?,"{'text': ['a computational problem', 'a comput...",a computational problem
1718,56e1a0dccd28a01900c67a2e,Computational_complexity_theory,It is tempting to think that the notion of fun...,How can function problems typically be restated?,"{'text': ['decision problems', 'as decision pr...",as decision problems


In [7]:
df.shape

(48, 6)

In [8]:
from datasets import Dataset
import time
# Function to evaluate as Llama index does not support async evaluation for HFInference API
def generate_responses(test_questions, pdf_file, test_answers):
  # outputs = [chat_bot.generate_response(q, pdf_file) for q in test_questions]
  outputs = []
  ri = 1
  for q in test_questions:
      try:
          outputs.append(chat_bot.generate_response(q, pdf_file))
          time.sleep(3)
          print('running No:',ri)
          ri += 1
      except:
          outputs.append({'answer': 'error', 'source_documents': []})
          time.sleep(3)
          print('Fail No:',ri)
          ri += 1
    

  answers = []
  contexts = []
  for output in outputs:
    # time.sleep(3)
    answers.append(output['answer'])
    contexts.append([doc.page_content for doc in output['source_documents']])
  dataset_dict = {
        "question": test_questions,
        "answer": answers,
        "contexts": contexts,
  }
  if test_answers is not None:
    dataset_dict["ground_truth"] = test_answers
  #ds = Dataset.from_dict(dataset_dict)
  return dataset_dict

In [9]:
import json
from src import chatbot

import time
path =  "D:\GithubLocal\RAG-with-Llama2\config.yaml"
pdf_path = "D:\GithubLocal\RAG-with-Llama2\eval\squad_comput_theory.pdf"
chat_bot = chatbot.PDFChatBot(config_path=path)

# begin

questions = df.loc[:]['question'].values.tolist()
test_answers = df.loc[:]['first_answer'].values.tolist()
dataset_dict = generate_responses(questions, pdf_path, test_answers)

output_file_path = f"outputs_squad.json"
with open(output_file_path, 'w') as json_file:
    json.dump(dataset_dict, json_file, indent=4)




d:\machineLearning\miniconda\envs\env_rag\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Computational complexity theory is a branch of the theory of computation in theoretical computer
science that focuses on classifying computational problems according to their inherent difficulty, and
relating those classes to each other. A computational problem is understood to be a task that is in
principle amenable to being solved by a computer, which is equivalent to stating that the problem may
be solved by mechanical application of mathematical steps, such as an algorithm.

Closely related fields in theoretical computer science are analysis of algorithms and computability
theory. A key distinction between analysis of algorithms and computational complexity theory is that
the former is devoted to analyzing the amount

d:\machineLearning\miniconda\envs\env_rag\lib\site-packages\transformers\generation\utils.py:1295: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
d:\machineLearning\miniconda\envs\env_rag\lib\site-packages\transformers\models\llama\modeling_llama.py:728: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(



> Finished chain.

> Finished chain.
running No: 1


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?
AI: Computational complexity theory.
Follow Up Input: What measure of a computational problem broadly defines the inherent difficulty of the solution?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

A problem is regarded as inherently difficult if its solution requires significant resources, whatever the
algorithm used

d:\machineLearning\miniconda\envs\env_rag\lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...


> Finished chain.

> Finished chain.
running No: 47


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What eponymous variation of arithmetic presents a decision problem not evidenced in P?
AI:  The eponymous variation of arithmetic that presents a decision problem not evidenced in P is Presburger arithmetic.
Human:What tactic did researchers employ to offset the former deficit of work surrounding the complexity of algorithmic problems?
AI:  Researchers employed the following tactics to offset the former deficit of work surrounding the complexity of algorithmic problems:
1. Definition of Turing machines by Alan Turing in 1936.
2. Imposing restrictions on the available resources.
3. Formalizing the intuition of inherently difficult problems by introducing mathematical models of computation to study these problems

d:\machineLearning\miniconda\envs\env_rag\lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In 1967, Manuel Blum developed an axiomatic complexity theory based on his axioms and proved an
important result, the so-called, speed-up theorem. The field really began to flourish in 1971 when the
US researcher Stephen Cook and, working independently, Leonid Levin in the USSR, proved that
there exist practically relevant problems that are NP-complete. In 1972, Richard Karp took this idea a
leap forward with his landmark paper, "Reducibility Among Combinatorial Problems", in which he

The time and space hierarchy theorems form the basis for most separation results of complexity
classes. For instance, the time hierarchy theorem tells us that P is strictly contained in EXPTIME, and
the space hierarchy t

d:\machineLearning\miniconda\envs\env_rag\lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.
running No: 48


In [10]:
dataset_dict

{'question': ['What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?',
  'What measure of a computational problem broadly defines the inherent difficulty of the solution?',
  'What two fields of theoretical computer science closely mirror computational complexity theory?',
  'What is the name given to the input string of a computational problem?',
  'By how many kilometers does the traveling salesman problem seek to classify a route between the 15 largest cities in Germany?',
  'In a computational problem, what can be described as a string over an alphabet?',
  'What kind of problems are one of the main topics studied in computational complexity theory?',
  'What kind of graph is an example of an input used in a decision problem?',
  'A function problem is an example of what?',
  'How can function problems typically be restated?',
  'What is a commonly used measurement used to determine the complexity 

In [11]:
ds = Dataset.from_dict(dataset_dict)

In [12]:
ds

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 48
})

In [15]:
df = ds.to_pandas()
df.to_csv('outputs_squad.csv', index=False)

In [21]:
with open('outputs_squad.json', 'w') as json_file:
    json.dump(dataset_dict, json_file, indent=4)

In [19]:
from apikeys import OPENAI_API
import pandas as pd
os.environ["OPENAI_API_KEY"] = OPENAI_API
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result = evaluate(
    ds.iloc[:24],
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

AttributeError: 'Dataset' object has no attribute 'iloc'

In [17]:
with open('result_squad.json', 'w') as json_file:
    json.dump(result, json_file, indent=4)

NameError: name 'result' is not defined

In [41]:
result.to_pandas()

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,How does RAG's technical integration with othe...,RAG's technical integration with other AI met...,[here Coral12are also gaining attention for th...,RAG's technical integration with other AI meth...,1.0,0.25,0.943303,1.0
1,How is contrastive learning used in the traini...,Contrastive learning is used in the training...,"[els, interaction pairs of input and output ar...",The initial phase of training focuses on the r...,1.0,1.00,0.947847,1.0
2,What are the modules involved in Advanced RAG?,The answer to this question is not provided ...,[focus on appraising the essential abilities o...,The modules involved in Advanced RAG are Searc...,1.0,NaN,0.000000,0.0
